In [1]:
import pandas as pd
from pandas import Series, DataFrame
import json
from janome.tokenizer import Tokenizer
from requests_oauthlib import OAuth1Session
from wordcloud import WordCloud
import emoji
import re
import csv

In [2]:
def create_oauth_session(oauth_key_dict):
    oauth = OAuth1Session(
        oauth_key_dict['consumer_key'],
        oauth_key_dict['consumer_secret'],
        oauth_key_dict['access_token'],
        oauth_key_dict['access_token_secret']
    )
    return oauth

def tweet(oauth, text):
    url = 'https://api.twitter.com/1.1/statuses/update.json'
    params = {'status': text + '\n#TweetFromPy'}
    req = oauth.post(url, params)

    if req.status_code == 200:
        print('tweet succeed!')
    else:
        print('tweet failed')
        
def tweet_image(oauth, text, image_file):
    url_media = "https://upload.twitter.com/1.1/media/upload.json"
    url_text = "https://api.twitter.com/1.1/statuses/update.json"
    
    files = {"media" : open(image_file, 'rb')}
    req_media = oauth.post(url_media, files = files)
    if req_media.status_code != 200:
        print ('media upload failed: %s', req_media.text)
        exit()
        
    media_id = json.loads(req_media.text)['media_id']
    print ("Media ID: %d" % media_id)
    
    params = {'status': text + '\n#TweetFromPy', 'media_ids': [media_id]}
    req = oauth.post(url_text, params)

    if req.status_code == 200:
        print('tweet succeed!')
    else:
        print('tweet failed')
        
def tweet_search(search_word, num, oauth):
    url = 'https://api.twitter.com/1.1/search/tweets.json'
    params = {
        'q': search_word,
        'count' : num,
        'result_type' : 'recent',
        'exclude': 'retweets',
        'lang' : 'ja'
        }
    responce = oauth.get(url, params=params)
    if responce.status_code != 200:
        print("Error code: %d" %(responce.status_code))
        return None
    tweets = json.loads(responce.text)
    return tweets

keysfile = '../../twitter_API/key/keys.json'
keys = json.load(open(keysfile))
oauth = create_oauth_session(keys)

In [3]:
def remove_emoji(text):
    return ''.join(c for c in text if c not in emoji.UNICODE_EMOJI['en'])

def remove_url(text):
    return re.sub(r'https?://[\w/:%#\$&\?\(\)~\.=\+\-]+', '', text)

In [4]:
search = tweet_search("ビットコイン", 100, oauth)
df_search = DataFrame.from_dict(search['statuses'])
tweets = df_search['text'].tolist()

In [5]:
tweets[:10]

['米S&amp;P、ビットコインとイーサの価格指数を開始\nhttps://t.co/Wqp4rOFNQ1\n#暗号資産 #仮想通貨 #ビットコイン #副業 #主婦 https://t.co/wLQDw20qO6',
 'ビットコインのターン！笑\n#BTC',
 'イーサの天井が390000あたりを想定してたので、利確ぴったりんこ\u3000ビットコイン現物に換えました！成功！0.08増えたよ！',
 'f7ee33be3aeb8485\n毎日ニュースを見るだけだビットコインがもらえる‼️\ncheeseアプリ登録時に入れたらビットコインもらえるよ！#cheese #ビットコイン',
 'ビットコインだけど400円くらい資産増えたｗｗｗ\nこのままずっと漬けておくんだぁ',
 '3/14？ぐらいから掘ってたビットコインが2万になったので現金化した',
 '金子勇/ファイル共有ソフト『Winny』の開発者として知られる情報工学者。著名な裁判「Winny事件」（ネットを扱うなら知っておいた方がいいだろう）にて無罪となるも、疲労が祟ったのか2013年に43歳で夭折。起業家の仲津正朗によれ… https://t.co/24cARFoHJM',
 'ビットコイン30万円下がってる！買い増しチャンス😇\n\n#投資家さんと繋がりたい\n#投資初心者 https://t.co/N4bJJHOA50',
 '有事のビットコインとなるか',
 'BitCin(ビットコイン)を買うならここ！\nCoinCheck⇒https://t.co/ksBjJp0haN']

In [6]:
texts = []
for tweet in tweets:
    text = tweet
    text = remove_emoji(text)
    text = remove_url(text)
    texts.append(text)

In [7]:
texts[:10]

['米S&amp;P、ビットコインとイーサの価格指数を開始\n\n#暗号資産 #仮想通貨 #ビットコイン #副業 #主婦 ',
 'ビットコインのターン！笑\n#BTC',
 'イーサの天井が390000あたりを想定してたので、利確ぴったりんこ\u3000ビットコイン現物に換えました！成功！0.08増えたよ！',
 'f7ee33be3aeb8485\n毎日ニュースを見るだけだビットコインがもらえる️\ncheeseアプリ登録時に入れたらビットコインもらえるよ！#cheese #ビットコイン',
 'ビットコインだけど400円くらい資産増えたｗｗｗ\nこのままずっと漬けておくんだぁ',
 '3/14？ぐらいから掘ってたビットコインが2万になったので現金化した',
 '金子勇/ファイル共有ソフト『Winny』の開発者として知られる情報工学者。著名な裁判「Winny事件」（ネットを扱うなら知っておいた方がいいだろう）にて無罪となるも、疲労が祟ったのか2013年に43歳で夭折。起業家の仲津正朗によれ… ',
 'ビットコイン30万円下がってる！買い増しチャンス\n\n#投資家さんと繋がりたい\n#投資初心者 ',
 '有事のビットコインとなるか',
 'BitCin(ビットコイン)を買うならここ！\nCoinCheck⇒']

In [8]:
from janome.tokenizer import Tokenizer

In [16]:
t = Tokenizer()
worddic = {}

In [17]:
for line in texts:
    malist = t.tokenize(line)
    for w in malist:
        word = w.surface
        part = w.part_of_speech
        if part.find('名詞') < 0:
            continue
        if not word in worddic:
            worddic[word] = 0
        worddic[word] += 1

In [18]:
trend_words = ''
keys = sorted(worddic.items(), key=lambda x:x[1], reverse=True)
for word, cnt in keys[:50]:
    trend_words += word + ' '
    print('{0}({1})'.format(word, cnt))

コイン(124)
ビット(121)
#(46)
通貨(40)
仮想(37)
円(31)
,(19)
万(17)
@(15)
BTC(13)
ニュース(13)
イーサリアム(13)
1(12)
((11)
－(11)
0(10)
の(10)
投資(10)
まとめ(10)
:(10)
$(10)
資産(9)
/(9)
)(9)
速報(9)
News(9)
毎日(8)
ん(8)
2(8)
000(8)
今(8)
価格(7)
暗号(7)
登録(7)
人(7)
米(6)
イーサ(6)
.(6)
アプリ(6)
時(6)
方(6)
取引(6)
_(6)
分(6)
無料(6)
開始(5)
さん(5)
送金(5)
4(5)
リップル(5)


In [11]:
def word_cloud(words, image_file, font_path='C:\Windows/Fonts/MSGOTHIC.TTC'):
    wordcloud = WordCloud(
            background_color='white', font_path=font_path,   ## 日本語フォントを指定
            width=800, height=400).generate(words)

    ## 結果をファイルへ保存
    wordcloud.to_file(image_file)

    return True

In [22]:
word_cloud(trend_words, 'word_cloud_test.png')

True

In [26]:
search_word = 'ビットコイン'
search_num = 10
trend_word_num = 50
tweet_text = 'search word: {}\nsearch tweets num: {}\ntrend words num: {}'.format(search_word, search_num, trend_word_num)
tweet_text

'search word: ビットコイン\nsearch tweets num: 10\ntrend words num: 50'

In [27]:
print(tweet_text)

search word: ビットコイン
search tweets num: 10
trend words num: 50


In [30]:
tweet_image(oauth, tweet_text, 'word_cloud_test.png')

Media ID: 1389200235184877569
tweet succeed!


In [29]:
def get_wakachi(list_text, word, hinshi=['名詞', '形容詞']):
    remove_words = ['こと', 'よう', 'そう', 'これ', 'それ', 'もの', 'ここ', 'さん', 
                'ところ', 'とこ', word]
    t = Tokenizer()
    wakachi = ''
    for text in list_text:
        malist = t.tokenize(text)
        for w in malist:
            word = w.surface
            part = w.part_of_speech
            hit = False
            for h in hinshi:
                hit = hit or (h in part)
            if not hit:
                continue
            if word not in remove_words:
                wakachi += word + ' '
    print(wakachi)
    return wakachi

In [30]:
wakachi = get_wakachi(texts, 'ビットコイン')
wakachi

米 S & amp ; P ビット コイン イーサ 価格 指数 開始 # 暗号 資産 # 仮想 通貨 # ビット コイン # 副業 # 主婦 ビット コイン ターン 笑 # BTC イーサ 天井 390000 あたり 想定 利 確 こ ビット コイン 現物 成功 0 . 08 f 7 ee 33 be 3 aeb 8485 毎日 ニュース ビット コイン cheese アプリ 登録 時 ビット コイン # cheese # ビット コイン ビット コイン 400 円 資産 ｗｗｗ まま ん ぁ 3 / 14 ビット コイン 2 万 現金 化 金子 勇 / ファイル 共有 ソフト Winny 開発 者 情報 工学 者 著名 裁判 Winny 事件 ネット 方 いい 無罪 疲労 の 2013 年 43 歳 夭折 起業 家 仲 津 正朗 ビット コイン 30 万 円 増し チャンス # 投資 家 # 投資 初心者 有事 ビット コイン BitCin ( ビット コイン ) CoinCheck ⇒ ビット コイン どこ CO 2 カーボン 銘柄 ビット コイン 価格 上昇 マイニング 二酸化炭素 排出 浮き彫り ドーブ ターン 先 ビット コイン 取引 一般 貨幣 送金 違い 送金 手数料 ビット コイン 移動 超 少額 費用 以外 手数料 の 通常 銀行 送金 場合 国内 送金 数 百 円 海外 送金 4 , 000 円 8 , 000 円 程度 仮想 通貨 定点 観測 ( 1 , 234 日 目 )】 BTC 2 , 654 , 791 円 ETH 4 , 946 , 621 円 XRP 2 , 062 , 269 円 XEM 2 , 486 , 833 円 ( 利 確済 み ) 積立 原資 各 39 万 円 ( 毎月 1 万 円 ) 毎日 自動 ビット コイン 下げ トレンド 継続 ビット コイン 下落 笑 5 万 幅 指値 笑 0 . 0003 btc v (￣ ー ￣) v @ konsone _ p 1 ビット コイン お願い 11 時 仮想 通貨 レート ビット コイン / 円 612 万 2482 円 ビット コイン FX / 円 659 万 8065 円 イーサリアム / 円 36 万 0982 円 リップル / 円 162 . 53 円 # zaifx 

'米 S & amp ; P ビット コイン イーサ 価格 指数 開始 # 暗号 資産 # 仮想 通貨 # ビット コイン # 副業 # 主婦 ビット コイン ターン 笑 # BTC イーサ 天井 390000 あたり 想定 利 確 こ ビット コイン 現物 成功 0 . 08 f 7 ee 33 be 3 aeb 8485 毎日 ニュース ビット コイン cheese アプリ 登録 時 ビット コイン # cheese # ビット コイン ビット コイン 400 円 資産 ｗｗｗ まま ん ぁ 3 / 14 ビット コイン 2 万 現金 化 金子 勇 / ファイル 共有 ソフト Winny 開発 者 情報 工学 者 著名 裁判 Winny 事件 ネット 方 いい 無罪 疲労 の 2013 年 43 歳 夭折 起業 家 仲 津 正朗 ビット コイン 30 万 円 増し チャンス # 投資 家 # 投資 初心者 有事 ビット コイン BitCin ( ビット コイン ) CoinCheck ⇒ ビット コイン どこ CO 2 カーボン 銘柄 ビット コイン 価格 上昇 マイニング 二酸化炭素 排出 浮き彫り ドーブ ターン 先 ビット コイン 取引 一般 貨幣 送金 違い 送金 手数料 ビット コイン 移動 超 少額 費用 以外 手数料 の 通常 銀行 送金 場合 国内 送金 数 百 円 海外 送金 4 , 000 円 8 , 000 円 程度 仮想 通貨 定点 観測 ( 1 , 234 日 目 )】 BTC 2 , 654 , 791 円 ETH 4 , 946 , 621 円 XRP 2 , 062 , 269 円 XEM 2 , 486 , 833 円 ( 利 確済 み ) 積立 原資 各 39 万 円 ( 毎月 1 万 円 ) 毎日 自動 ビット コイン 下げ トレンド 継続 ビット コイン 下落 笑 5 万 幅 指値 笑 0 . 0003 btc v (￣ ー ￣) v @ konsone _ p 1 ビット コイン お願い 11 時 仮想 通貨 レート ビット コイン / 円 612 万 2482 円 ビット コイン FX / 円 659 万 8065 円 イーサリアム / 円 36 万 0982 円 リップル / 円 162 . 53 円 # zaifx

# ネガポジ

In [12]:
np_dic = {}
fp = open("../../lib/pn.csv", "rt", encoding="utf-8")
reader = csv.reader(fp, delimiter='\t')
for i, row in enumerate(reader):
  name = row[0]
  result = row[1]
  np_dic[name] = result
  if i % 500 == 0: print(i)
print("ok")

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
ok


In [14]:
print(np_dic["笑顔"])
print(np_dic["嫌い"])
print(np_dic["時間"])

p
n
e


In [23]:
for word, cnt in keys[:50]:
    if word not in np_dic:
        print(word, end=' ')
    elif np_dic[word] == 'p':
        print(word, end='(p) ')
    elif np_dic[word] == 'n':
        print(word, end='(n) ')
    elif np_dic[word] == 'e':
        print(word, end=' ')

コイン(p) ビット # 通貨 仮想 円 , 万 @ BTC ニュース イーサリアム 1 ( － 0 の 投資 まとめ : $ 資産(p) / ) 速報 News 毎日 ん 2 000 今 価格(n) 暗号 登録 人 米 イーサ . アプリ 時 方 取引(n) _ 分 無料(p) 開始 さん 送金 4 リップル 